In [1]:
%reload_ext autoreload
%autoreload 1
%aimport train

In [2]:
import pandas as pd
import numpy as np

In [3]:
num_docs = 10000

tweet_df = pd.read_pickle('tweet_data_tokenized.pkl')
tweet_df = tweet_df.sort_values(by='id', ascending=False)
tweet_df = tweet_df.head(num_docs)
tweet_df = tweet_df.reset_index()

In [4]:
import pickle

with open('tweet_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [5]:
def build_context(v, context_size=3):
    k = context_size
    v = np.lib.pad(v, (k,0), 'constant', constant_values=(0,))
    for i in range(len(v) - k):
        yield v[i:i+k],v[i+k]

data = [(tweet_id, context, target) for tweet_id, words in tweet_df.tokenized.iteritems() for context, target in build_context(words)]

docs     = np.array([x[0] for x in data])
contexts = np.array([x[1] for x in data])
targets  = np.array([x[2] for x in data])

In [6]:
import tensorflow as tf
import tflearn

In [7]:
g = tf.Graph()

In [8]:
num_words = len(vocab)
doc_embedding_size = 10
word_embedding_size = 50

with g.as_default():
    input_layer1 = tflearn.input_data(shape=[None, 1])
    input_layer2 = tflearn.input_data(shape=[None, 3])

    embedding_layer1, = tflearn.embedding(input_layer1, input_dim=num_docs, output_dim=doc_embedding_size)
    e1,e2,e3 = tflearn.embedding(input_layer2, input_dim=num_words, output_dim=word_embedding_size)

    embedding_layer = tflearn.merge([embedding_layer1, e1, e2, e3], mode='concat')
    
    softmax = tflearn.fully_connected(embedding_layer, num_words, activation='softmax')

    optimizer = tflearn.optimizers.Adam(learning_rate=0.001)
    metric = tflearn.metrics.Accuracy()
    net = tflearn.regression(softmax, optimizer=optimizer, metric=metric, batch_size=32,
                             loss='categorical_crossentropy')

In [9]:
with g.as_default():
    model = tflearn.DNN(net, tensorboard_verbose=0)

In [10]:
with g.as_default():
    X1 = docs[:,np.newaxis]
    X2 = contexts

    Y = tflearn.data_utils.to_categorical(targets, num_words)
    model.fit([X1, X2], Y, n_epoch=20, show_metric=True, run_id="dense_model")

Training Step: 7628  | total loss: 5.15014
| Adam | epoch: 001 | loss: 5.15014 - acc: 0.2774 -- iter: 078144/165934


KeyboardInterrupt: 

In [13]:
targets

array([8344, 6705,    1, ...,  263, 4469,  154])